In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
DATA_FOLDER = "../data"

LABEL_TRAIN = Path(DATA_FOLDER, "train.csv")
LABEL_VAL = Path(DATA_FOLDER, "validation.csv")
LABEL_SUB = Path(DATA_FOLDER, "submission_sample.csv")

In [ ]:
df = pd.read_csv(LABEL_TRAIN)
df

In [ ]:
unique_filenames = df.filename.unique()

# Print the 5 first unique filenames
unique_filenames[:5]

In [ ]:
for x in unique_filenames:
    if "bGaslniO4a" in x:
        print(x)

In [ ]:
import tifffile
from PIL import Image, ImageDraw


def draw_rectangles_on_tiff(image_path: Path, csv_file: Path, page_number: int = 0):
    print("Working on:", image_path, "\n")

    # Open the TIFF file using tifffile
    with tifffile.TiffFile(image_path) as tif:
        # Read the specified page
        max_res_shape = tif.pages[0].shape
        page = tif.pages[page_number]
        page_shape = page.shape
        ratio = 2**page_number

        print("Original shape:", max_res_shape)
        print(f"Page shape n°{page_number}:", page_shape)
        print(
            "Ratio max_res/page:",
            (max_res_shape[0] / page_shape[0], max_res_shape[1] / page_shape[1]),
            "\n",
        )

        image_array = page.asarray()

        # Convert the image array to a PIL image
        image = Image.fromarray(image_array)
        draw = ImageDraw.Draw(image)

        df = pd.read_csv(csv_file)
        df = df[df.filename == image_path.name]
        print(df.head())
        print()

        # Assuming the CSV file has columns named 'x1', 'y1', 'x2', 'y2' for the rectangles
        for i, row in df.iterrows():
            box = (row["x1"], row["y1"], row["x2"], row["y2"])
            print(f"Box {i} - Coords (max res):", *box)
            print("Max res:", *box)
            box = tuple(element / ratio for element in box)
            print("Resized:", *box)

            # Draw rectangle with red outline
            draw.rectangle(box, outline="red", width=30)

        # Save or show the image
        image.show()


# Example usage

PAGE_IDX = 5
img_path = Path(DATA_FOLDER, "images", "bGaslniO4a_a.tif")

draw_rectangles_on_tiff(img_path, LABEL_TRAIN, PAGE_IDX)
